In [ ]:
################################ Read in JSON ###########################
import json

with open('kaggle_train.json') as data_file:
    data_train = json.load(data_file)

In [101]:
##################### Convert json to data frame #######################
import pandas as pd

df = pd.io.json.json_normalize(data_train)    #convert to data frame
df.columns = ["cuisine","ID","ingredients"]   #rename columns of data frame
#df = df.sort(columns="ID")                    #sort by ID ?
df = df.set_index(df["ID"])                   #set ID as index
del df["ID"]                                  #delete previous ID
#df["cuisine"] = df["cuisine"].astype("category")   #categorize cuisines ?
df["count_ingredients"] = [len(l) for l in df["ingredients"]]  #add a count ingredients columns
#df.groupby("cuisine").count()
#print df.head()

In [111]:
############################## Stemming and Lematizing ###################
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

def stemmer(text,stemmer=LancasterStemmer()):
    return stemmer.stem(text).lower()


    
df["ingredients"] = [[" ".join(stemmer(word) for word in x.split()) for x in l] for l in df["ingredients"]]  #stem each word

maxim in a bal


In [127]:
df.groupby("cuisine").boxplot()

OrderedDict([(u'brazilian', {'boxes': [<matplotlib.lines.Line2D object at 0x000000000DF5A898>], 'fliers': [<matplotlib.lines.Line2D object at 0x000000000DF72470>], 'medians': [<matplotlib.lines.Line2D object at 0x000000000DF66EB8>], 'means': [], 'whiskers': [<matplotlib.lines.Line2D object at 0x000000000DF5A9E8>, <matplotlib.lines.Line2D object at 0x000000000DF5AE10>], 'caps': [<matplotlib.lines.Line2D object at 0x000000000DF663C8>, <matplotlib.lines.Line2D object at 0x000000000DF66940>]}), (u'british', {'boxes': [<matplotlib.lines.Line2D object at 0x000000000DF72CF8>], 'fliers': [<matplotlib.lines.Line2D object at 0x000000002972C9E8>], 'medians': [<matplotlib.lines.Line2D object at 0x000000002972C470>], 'means': [], 'whiskers': [<matplotlib.lines.Line2D object at 0x000000000DF72F60>, <matplotlib.lines.Line2D object at 0x00000000297213C8>], 'caps': [<matplotlib.lines.Line2D object at 0x0000000029721940>, <matplotlib.lines.Line2D object at 0x0000000029721EB8>]}), (u'cajun_creole', {'box

In [124]:
import nltk
def extract_nouns(text):
    return " ".join([word \
                     for word,tag in nltk.pos_tag(nltk.word_tokenize(text)) \
                     if(tag=="NN" or tag=="NNP" or tag=="NNS" or tag=="NNPS")])

st = "Maximum in a baLl"

#print extract_nouns(st)

st = " ".join([stemmer(w).lower() for w in st.split()])
print st

LookupError: 
**********************************************************************
  Resource u'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\DELL/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\DELL\\Anaconda\\nltk_data'
    - 'C:\\Users\\DELL\\Anaconda\\lib\\nltk_data'
    - 'C:\\Users\\DELL\\AppData\\Roaming\\nltk_data'
    - u''
**********************************************************************

In [125]:
nltk.download()

showing info http://www.nltk.org/nltk_data/


True

In [116]:
############################### Store the precious data frame #############################
import pickle
pickle.dump(df,open("data_train_data_frame.p","wb"))

In [118]:
################################ Make Ingredients and cuisines lists and dictionaries ##################################
data = data_train
X = []
Y = []
for k in data:
    X.extend(list(k["ingredients"]))
    Y.append(k["cuisine"])
print X[1:10]

X=set(X)
Y=set(Y)

print list(X)[1:10]
ingredients={}
cuisine={}
k=0
for x in X:
    ingredients[x]=k
    k=k+1
k=0
for y in Y:
    cuisine[y]=k
    k=k+1
print(len(data)),type(data),type(data[1]),data[1]
print(len(ingredients)),type(ingredients)
print(len(cuisine)),type(cuisine)
print(cuisine)

[u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles', u'plain flour']
[u'sweetened coconut', u'baking chocolate', u'egg roll wrappers', u'bottled low sodium salsa', u'vegan parmesan cheese', u'clam sauce', u'mahlab', u'(10 oz.) frozen chopped spinach, thawed and squeezed dry', u'figs']
39774 <type 'list'> <type 'dict'> {u'cuisine': u'southern_us', u'id': 25693, u'ingredients': [u'plain flour', u'ground pepper', u'salt', u'tomatoes', u'ground black pepper', u'thyme', u'eggs', u'green tomatoes', u'yellow corn meal', u'milk', u'vegetable oil']}
6714 <type 'dict'>
20 <type 'dict'>
{u'irish': 0, u'mexican': 1, u'chinese': 2, u'filipino': 3, u'vietnamese': 4, u'spanish': 13, u'japanese': 7, u'moroccan': 5, u'french': 12, u'greek': 9, u'indian': 10, u'jamaican': 11, u'british': 8, u'brazilian': 6, u'russian': 14, u'cajun_creole': 15, u'thai': 16, u'southern_us': 17, u'korean': 18, u'italian': 19}


In [2]:
###############################################################################
##########Setting Training and Testing Data####################################
###############################################################################
import pickle
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
#data = data[:100]

data_train,data_test=train_test_split(data,test_size=0.3,random_state=1234)
data_cv,data_test=train_test_split(data_test,test_size=0.5,random_state=1234)
print("yyyyyyyyyyyyy")
print(len(data_train))
features_train=[]
label_train=[]

for k in data_train:
    x=[0 for x in range(0,len(ingredients))]
    for k2 in k["ingredients"]:
        x[ingredients[k2]]=1
    features_train.append(x)
    label_train.append(cuisine[k["cuisine"]])

features_cv=[]
label_cv=[]

for k in data_cv:
    x=[0 for x in range(0,len(ingredients))]
    for k2 in k["ingredients"]:
        x[ingredients[k2]]=1
    features_cv.append(x)
    label_cv.append(cuisine[k["cuisine"]])

features_test=[]
label_test=[]

for k in data_test:
    x=[0 for x in range(0,len(ingredients))]
    for k2 in k["ingredients"]:
        x[ingredients[k2]]=1
    features_test.append(x)
    label_test.append(cuisine[k["cuisine"]])

print(len(features_train[0]))
print(len(features_cv[0]))

yyyyyyyyyyyyy
27841
6714
6714


In [3]:
p = PCA(n_components=2000)

In [7]:
x_train = p.fit_transform(features_train)
pickle.dump(p,open("PCA_object.p","wb"))
pickle.dump(p,open("PCA_x_train.p","wb"))

In [5]:
p = pickle.load(open("PCA_object.p","rb"))
x_train = pickle.load(open("PCA_x_train.p","rb"))

EOFError: 

In [8]:
x_cv = p.transform(features_cv)
x_test = p.transform(features_test)

In [3]:
###############################################################################
##########Setting Training and Testing Data####################################
###############################################################################


from sklearn import tree
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

def classify(model,name):
    model = Pipeline([('feature_selection',LinearSVC(penalty="l1",dual=False)),('classification',model)])
    model.fit(features_train,label_train)
#    print("Fitted")
#    predicted= model.predict(features_cv)
    acc = round(model.score(features_cv,label_cv),4)*100
    print("Accuracy for ",name," : ",acc)
    file_k.write(str(name)+","+str(acc)+"\n")
    return acc,clf

print "done"

done


In [4]:
clfs = [
        #(AdaBoostClassifier((DecisionTreeClassifier(max_depth=3)),n_estimators=100,learning_rate=1),"ADA 1")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=5)),n_estimators=500,learning_rate=1),"ADA 2")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=5)),n_estimators=1000,learning_rate=1),"ADA 3")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=5)),n_estimators=1400,learning_rate=1),"ADA 4")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=7)),n_estimators=1200,learning_rate=1),"ADA 5")
        #,(AdaBoostClassifier((DecisionTreeClassifier(max_depth=9)),n_estimators=1200,learning_rate=1),"ADA 6")
        #,(tree.DecisionTreeClassifier(),"Decision Tree")
        #,(RandomForestClassifier(n_estimators=500,n_jobs=20),"Random Forests 1")
        (linear_model.LogisticRegression(C=1e5),"Logistic Regression")
        #,(LinearSVC(loss='l2',penalty='l1',dual=False,tol=1e-3),"SVM1")
        #,(LinearSVC(loss='l2',penalty='l2',dual=False,tol=1e-3),"SVM2")
        ]
#model.fit(features,label)

In [5]:
file_log_classifiers_name = ""
for clf,name in clfs:
    file_log_classifiers_name += name+"_"
file_log_classifiers_name += ".txt"
file_k=open(file_log_classifiers_name,"w")
print file_log_classifiers_name

Logistic Regression_.txt


In [6]:
acc = 0
for i in range(1,2):
    print "Episode :",i
    file_k.write("\n\n\nIteration : "+str(i)+"\n\n")
    for clf,name in clfs:
        a,c = classify(clf,name)
        if a>acc:
            print "best clf:",name
            pickle.dump(c,open("best_classifier.p","wb"))
            pickle.dump(name,open("best_classifier_name.p","wb"))
    print("\n\n\n")
file_k.close()

print("Now on with testing!! The moment of truth! Dil thaam ke bethiye.")

clf = pickle.load(open("best_classifier.p","rb"))
name = pickle.load(open("best_classifier_name.p","rb"))
#predicted= clf.predict(x)
acc = round(clf.score(features_test,label_test),4)*100
print("Accuracy for ",name," : ",acc)
file_k.write("\nBest classifier : "+str(name)+':'+str(acc))
#predicted= model.predict(test_data)
#for k in predicted:
#    print(k, end=" ")
#print()
#print("Prediction Score")
#print()
file_k.close()

Episode : 1
('Accuracy for ', 'Logistic Regression', ' : ', 63.41)
best clf: Logistic Regression




Now on with testing!! The moment of truth! Dil thaam ke bethiye.


ValueError: X has 6714 features per sample; expecting 4752